In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from camel.logger import get_logger
from camel.datasets import StaticDataset, FewShotGenerator


logger = get_logger(__name__)



data = [
    {
        "question": "Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).",
        "final_answer": "0",
        "rationale": '''from sympy import symbols, Eq, solve
a, b = symbols('a b')
eq1 = Eq(2*a + 3, -3)
eq2 = Eq(-7, -4 - b)
sol_a = solve(eq1, a)[0]
sol_b = solve(eq2, b)[0]
result = sol_a + sol_b
print(result)'''
    },
    {
        "question": "What is the degree of the polynomial $(4 +5x^3 +100 +2\\pi x^4 + \\sqrt{10}x^4 +9)$?",
        "final_answer": "4",
        "rationale": '''from sympy import symbols, pi, sqrt, degree
x = symbols('x')
poly = 4 + 5*x**3 + 100 + 2*pi*x**4 + sqrt(10)*x**4 + 9
deg = degree(poly, x)
print(deg)'''
    },
    {
        "question": "Evaluate $\\left\\lceil3\\left(6-\\frac12\\right)\\right\\rceil$.",
        "final_answer": "17",
        "rationale": '''from sympy import Rational, ceiling
expr = 3 * (6 - Rational(1,2))
result = ceiling(expr)
print(result)'''
    },
    {
        "question": "Sam is hired for a 20-day period. On days that he works, he earns $\\$$60. For each day that he does not work, $\\$$30 is subtracted from his earnings. At the end of the 20-day period, he received $\\$$660. How many days did he not work?",
        "final_answer": "6",
        "rationale": '''from sympy import symbols, Eq, solve
x, y = symbols('x y')
eq1 = Eq(x + y, 20)
eq2 = Eq(60*x - 30*y, 660)
solution = solve((eq1, eq2), (x, y))
print(solution[y])'''
    },
    {
        "question": "Find the center of the circle with equation $x^2 - 6x + y^2 + 2y = 9$.",
        "final_answer": "(3, -1)",
        "rationale": '''from sympy import symbols, Eq, solve
x, y = symbols('x y')
# Complete the square for x: x^2 - 6x = (x - 3)^2 - 9
# Complete the square for y: y^2 + 2y = (y + 1)^2 - 1
# Equation becomes: (x - 3)^2 - 9 + (y + 1)^2 - 1 = 9
# Simplify: (x - 3)^2 + (y + 1)^2 = 19
h = 3  # x-coordinate of center
k = -1  # y-coordinate of center
result = (h, k)
print(result)'''
    },
    {
        "question": "What are all values of $p$ such that for every $q>0$, we have $$\\frac{3(pq^2+p^2q+3q^2+3pq)}{p+q}>2p^2q?$$ Express your answer in interval notation in decimal form.",
        "final_answer": "[0, 3)",
        "rationale": '''from sympy import symbols, solve, GreaterThan
p = symbols('p')
# After simplification: 3(p + 3) > 2*p**2 for all q > 0, assuming p >= 0
expr = 2*p**2 - 3*p - 9  # Solve 2p^2 - 3p - 9 < 0
solutions = solve(expr < 0, p)
# Roots are -1.5 and 3, negative between them, intersect with p >= 0
result = "[0, 3)"  # Hardcoded interval based on solution and p >= 0
print(result)'''
    },
    {
        "question": "If $x = 2$ and $y = 5$, then what is the value of $\\frac{x^4+2y^2}{6}$ ?",
        "final_answer": "11",
        "rationale": '''from sympy import symbols
x, y = symbols('x y')
expr = (x**4 + 2*y**2) / 6
result = expr.subs({x: 2, y: 5})
print(result)'''
    },
    {
        "question": "Tim wants to invest some money in a bank which compounds quarterly with an annual interest rate of $7\\%$. To the nearest dollar, how much money should he invest if he wants a total of $\\$60,\\!000$ at the end of $5$ years?",
        "final_answer": "42409",
        "rationale": '''from sympy import symbols, floor
P = symbols('P')
A = 60000
r = 0.07
n = 4
t = 5
eq = Eq(A, P * (1 + r/n)**(n*t))
P_value = solve(eq, P)[0]
result = floor(P_value.evalf())  # Round down to nearest dollar
print(result)'''
    },
    {
        "question": "Berengere and her American foreign-exchange student Emily are at a bakery in Paris that accepts both euros and American dollars. They want to buy a cake, but neither of them has enough money. If the cake costs 6 euros and Emily has an American five-dollar bill, how many euros does Berengere need to contribute to the cost of the cake if 1 euro = 1.25 USD?",
        "final_answer": "2",
        "rationale": '''from sympy import symbols, Eq, solve
B = symbols('B')
# 1 euro = 1.25 USD, so 5 USD = 5 / 1.25 euros
emily_contribution = 5 / 1.25  # 4 euros
total_cost = 6
eq = Eq(emily_contribution + B, total_cost)
result = solve(eq, B)[0]
print(int(result))'''
    },
    {
        "question": "Simplify $\\sqrt[3]{1+8} \\cdot \\sqrt[3]{1+\\sqrt[3]{8}}$.",
        "final_answer": "3",
        "rationale": '''from sympy import cbrt
# sqrt[3]{1 + 8} = cbrt(9)
# sqrt[3]{8} = 2, so sqrt[3]{1 + sqrt[3]{8}} = cbrt(1 + 2) = cbrt(3)
result = cbrt(9) * cbrt(3)  # cbrt(9 * 3) = cbrt(27) = 3
print(result)'''
    }
]

seed_dataset = StaticDataset(data)

/home/student/Code/camel/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from camel.verifiers import PythonVerifier

verifier = PythonVerifier(required_packages=["sympy"])
await verifier.setup(uv=True)

In [4]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig


model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig().as_dict(),
)
generator = FewShotGenerator(
    seed_dataset=seed_dataset, verifier=verifier, model=model
)

In [5]:
await generator.generate_new(n=10)

[DataPoint(question='What is the derivative of the function $f(x) = 3x^4 - 5x^2 + 7$?', final_answer='12*x**3 - 10*x', rationale="from sympy import symbols, diff\nx = symbols('x')\nf = 3*x**4 - 5*x**2 + 7\nderivative = diff(f, x)\nprint(derivative)", metadata={'synthetic': 'True', 'created': '2025-03-28T12:43:56.867630', 'generator': 'few_shot'}),
 DataPoint(question='Aisha is collecting stamps over a 30-day period. For each day that she collects stamps, she earns $\\$50$. If she skips a day, she loses $\\$20$. At the end of the 30 days, Aisha has a total of $\\$1000$. How many days did she skip?', final_answer='50/7', rationale="from sympy import symbols, Eq, solve\nx, y = symbols('x y')\neq1 = Eq(x + y, 30)\neq2 = Eq(50*x - 20*y, 1000)\nsolution = solve((eq1, eq2), (x, y))\nprint(solution[y])", metadata={'synthetic': 'True', 'created': '2025-03-28T12:44:00.435510', 'generator': 'few_shot'}),
 DataPoint(question='If a car travels 60 miles in 1.5 hours, what is its average speed in mil

In [6]:
generator.save_to_jsonl(file_path="data.jsonl")

In [7]:
from pathlib import Path

proposal_synthetic_data = StaticDataset(Path("data.jsonl"))

In [8]:
from camel.agents import ChatAgent

agent = ChatAgent(model=model, system_message= "Wrap your answer in \\boxed{...}. Use chain of thought reasoning to come up with a solution to the following question: ")

In [11]:
cot_solutions = []

for i in range(len(proposal_synthetic_data)):
    dp = proposal_synthetic_data[i]
    question = dp.question

    agent_output = agent.step(question).msgs[0]
    cot_solutions.append(agent_output)
    print(agent_output)
    agent.reset()

BaseMessage(role_name='Assistant', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content="To find the derivative of the function \\( f(x) = 3x^4 - 5x^2 + 7 \\), we will apply the power rule for differentiation. The power rule states that if \\( f(x) = ax^n \\), then the derivative \\( f'(x) = a \\cdot n \\cdot x^{n-1} \\).\n\nLet's differentiate each term of the function \\( f(x) \\):\n\n1. For the first term \\( 3x^4 \\):\n   \\[\n   \\frac{d}{dx}(3x^4) = 3 \\cdot 4 \\cdot x^{4-1} = 12x^3.\n   \\]\n\n2. For the second term \\( -5x^2 \\):\n   \\[\n   \\frac{d}{dx}(-5x^2) = -5 \\cdot 2 \\cdot x^{2-1} = -10x.\n   \\]\n\n3. For the constant term \\( 7 \\):\n   \\[\n   \\frac{d}{dx}(7) = 0.\n   \\]\n\nNow we can combine the derivatives of each term:\n\\[\nf'(x) = 12x^3 - 10x + 0.\n\\]\n\nThus, simplifying gives:\n\\[\nf'(x) = 12x^3 - 10x.\n\\]\n\nTherefore, the derivative of the function \\( f(x) = 3x^4 - 5x^2 + 7 \\) is \n\\[\n\\boxed{12x^3 - 10x}.\n\\]", video_bytes=None, im

In [35]:
from camel.extractors import BaseExtractor, BoxedStrategy

answers = [cs.content for cs in cot_solutions]
gts = [proposal_synthetic_data[i].final_answer for i in range(len(proposal_synthetic_data))]

extractor = BaseExtractor([[BoxedStrategy()]])

await extractor.setup()

verifiable_ans = []

for ans in answers:
    verifiable_ans.append(await extractor.extract(ans))

print(verifiable_ans)


cot_verifier = PythonVerifier(required_packages=["sympy"])
await cot_verifier.setup(uv=True)

res = await verifier.verify_batch(verifiable_ans, gts)

['12x^3 - 10x', '14', '40', '6', '-10', '3', '-11', '8', '340', '2']


In [36]:
res

[VerificationResult(status=<VerificationOutcome.ERROR: 'error'>, result='', duration=0.011548757553100586, timestamp=datetime.datetime(2025, 3, 28, 13, 1, 11, 789231), metadata={'attempt': 1}, error_message='Solution code error:\nFile "/tmp/tmp6cd3vwvb.py", line 1\n    12x^3 - 10x\n     ^\nSyntaxError: invalid decimal literal'),
 VerificationResult(status=<VerificationOutcome.ERROR: 'error'>, result='', duration=9.989738464355469e-05, timestamp=datetime.datetime(2025, 3, 28, 13, 1, 11, 789631), metadata={'attempt': 1}, error_message='Ground truth evaluation error: malformed node or string on line 1: <ast.BinOp object at 0x768cf03e8dc0>'),
 VerificationResult(status=<VerificationOutcome.SUCCESS: 'success'>, result='40', duration=4.696846008300781e-05, timestamp=datetime.datetime(2025, 3, 28, 13, 1, 11, 789731), metadata={'attempt': 1}, error_message=None),
 VerificationResult(status=<VerificationOutcome.SUCCESS: 'success'>, result='6', duration=4.220008850097656e-05, timestamp=datetime.